GPU 연결을 추천드립니다!

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers


(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 데이터 전처리
x_train= x_train / 255.0  # 0~255 값을 0~1로 정규화
x_test = x_test / 255.0

# 레이블을 원-핫 인코딩
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 10) (10000, 32, 32, 3) (10000, 10)


In [ ]:
import tensorflow as tf


class VGGNet(tf.keras.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = self.conv_block(64)
        self.conv2 = self.conv_block(128)
        self.conv3 = self.conv_block(256)

        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(1024, activation='relu')
        self.fc2 = layers.Dense(num_classes, activation='softmax')

    def conv_block(self, filters):
        return models.Sequential([
            layers.Conv2D(filters, kernel_size=3, padding='same', activation='relu'),
            layers.Conv2D(filters, kernel_size=3, padding='same', activation='relu'),
            layers.MaxPool2D()
        ])

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

vgg_ = VGGNet(num_classes=10)
vgg_(tf.random.normal([1,32, 32, 3]))
vgg_.summary()

Model: "vgg_net_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_49 (Sequential)           │ (1, 16, 16, 64)             │          38,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_50 (Sequential)           │ (1, 8, 8, 128)              │         221,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_51 (Sequential)           │ (1, 4, 4, 256)              │         885,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (1, 4096)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (1, 1024)                   │       4,195,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_36 (Dense)                     │ (1, 10)                     │          10,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,350,986 (20.41 MB)

 Trainable params: 5,350,986 (20.41 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
vgg_.compile(optimizer='sgd',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

vgg_.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.1486 - loss: 2.2491 - val_accuracy: 0.2828 - val_loss: 2.0590
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.3339 - loss: 1.8549 - val_accuracy: 0.3994 - val_loss: 1.6649
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.4316 - loss: 1.5892 - val_accuracy: 0.4237 - val_loss: 1.6018
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.4842 - loss: 1.4480 - val_accuracy: 0.4869 - val_loss: 1.4335
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.5203 - loss: 1.3489 - val_accuracy: 0.4303 - val_loss: 1.6184


In [ ]:
import tensorflow as tf


class ResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.input_conv = layers.Conv2D(64, kernel_size=7, padding='same')

        self.conv1 = self.conv_block(64)
        self.conv2 = self.conv_block(128)
        self.conv3 = self.conv_block(256)

        self.convert1 = layers.Conv2D(64, kernel_size=1, strides=2, padding='same')
        self.convert2 = layers.Conv2D(128, kernel_size=1, strides=2, padding='same')
        self.convert3 = layers.Conv2D(256, kernel_size=1, strides=2, padding='same')

        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(1024, activation='relu')
        self.fc2 = layers.Dense(num_classes, activation='softmax')

    def conv_block(self, filters):
        return models.Sequential([
            layers.Conv2D(filters, kernel_size=3, strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', activation='relu'),
        ])

    def call(self, inputs):
        x = self.input_conv(inputs)

        res = x
        x = self.conv1(x)
        res = self.convert1(res)
        res = layers.Add()([res, x])

        x = self.conv2(x)
        res = self.convert2(res)
        res = layers.Add()([res, x])

        x = self.conv3(res)
        res = self.convert3(res)
        res = layers.Add()([res, x])

        x = self.flatten(res)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

resnet_ = ResNet(num_classes=10)
resnet_(tf.random.normal([1, 32, 32, 3]))
resnet_.summary()

Model: "res_net_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_177 (Conv2D)                  │ (1, 32, 32, 64)             │           9,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_73 (Sequential)           │ (1, 16, 16, 64)             │          74,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_74 (Sequential)           │ (1, 8, 8, 128)              │         221,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_75 (Sequential)           │ (1, 4, 4, 256)              │         886,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_184 (Conv2D)                  │ (1, 16, 16, 64)             │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_185 (Conv2D)                  │ (1, 8, 8, 128)              │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_186 (Conv2D)                  │ (1, 4, 4, 256)              │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (1, 4096)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_51 (Dense)                     │ (1, 1024)                   │       4,195,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_52 (Dense)                     │ (1, 10)                     │          10,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,442,890 (20.76 MB)

 Trainable params: 5,441,994 (20.76 MB)

 Non-trainable params: 896 (3.50 KB)

In [ ]:
resnet_.compile(optimizer='sgd',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

resnet_.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.2480 - loss: 2.0717 - val_accuracy: 0.3059 - val_loss: 1.9622
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.4133 - loss: 1.6612 - val_accuracy: 0.4011 - val_loss: 1.7303
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.4705 - loss: 1.4887 - val_accuracy: 0.4582 - val_loss: 1.5145
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.5174 - loss: 1.3683 - val_accuracy: 0.5157 - val_loss: 1.3708
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.5551 - loss: 1.2679 - val_accuracy: 0.4653 - val_loss: 1.5214


In [ ]:
from tensorflow.keras.applications import ResNet50

resnet = ResNet50(
    include_top=False,
    input_shape=(32, 32, 3),
    pooling='avg',
    classes=10
)

resnet.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_43            │ (None, 32, 32, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 38, 38, 3)      │              0 │ input_layer_43[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 16, 16, 64)     │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 16, 16, 64)     │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 16, 16, 64)     │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 18, 18, 64)     │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 8, 8, 64)       │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 8, 8, 64)       │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 8, 8, 64)       │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 8, 8, 64)       │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 8, 8, 64)       │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 8, 8, 64)       │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 8, 8, 64)       │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 8, 8, 256)      │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 8, 8, 256)      │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
resnet.trainable = True

resnet_clf = models.Sequential([
    resnet,
    layers.Dense(1024, activation='relu'),
    layers.Dense(10, activation='softmax')
])

resnet_clf.compile(optimizer='sgd',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

resnet_clf.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 112s 62ms/step - accuracy: 0.4693 - loss: 1.5831 - val_accuracy: 0.3464 - val_loss: 2.0215
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 45s 32ms/step - accuracy: 0.7627 - loss: 0.6874 - val_accuracy: 0.7437 - val_loss: 0.7769
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 42s 33ms/step - accuracy: 0.8346 - loss: 0.4845 - val_accuracy: 0.7556 - val_loss: 0.7458
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 31ms/step - accuracy: 0.8863 - loss: 0.3332 - val_accuracy: 0.7730 - val_loss: 0.7531
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 30ms/step - accuracy: 0.9193 - loss: 0.2321 - val_accuracy: 0.7347 - val_loss: 0.9339
